In [1]:
from __future__ import unicode_literals, print_function, division

import pickle
from typing import List

import pandas as pd

import time

import torch
import torch.nn as nn
from torch import optim

from io import open
import re
import random

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, Dataset
from torch.nn.utils.rnn import pad_sequence

from sklearn.model_selection import train_test_split

from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction
import nltk

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
dataset = pd.read_csv('../data/data_tokenize.csv')
pairs = list(dataset[["title", "text"]].itertuples(index=False, name=None))
train_pairs, val_pairs = train_test_split(pairs, test_size=0.1, random_state=42)

In [5]:
sos_token = 0
eos_token = 1
MAX_VOCAB_SIZE = 30_000

MAX_INPUT_LEN = 300
MAX_TARGET_LEN = 30


# Работа с данными

## Словарь частот

In [6]:
class Vocab:
    """Создаёт словари с частотами слов на основе входных данных"""

    def __init__(self, name):
        self.name = name
        self.word2index = {"<pad>": 0, "<unk>": 1, "<sos>": 2, "<eos>": 3}
        self.word2count = {"<pad>": 0, "<unk>": 0, "<sos>": 0, "<eos>": 0}
        self.index2word = {0: "<pad>", 1: "<unk>", 2: "<sos>", 3: "<eos>"}
        self.n_words = 4

        self._temp_word_counts = {}

    def addText(self, text: str):
        """Для каждого слова в тексте добавляет его во временный счётчик"""
        for word in text.split():
            self._temp_word_counts[word] = self._temp_word_counts.get(word, 0) + 1

    def build_vocab(self, is_text: bool = False):
        """Строит финальный словарь после подсчёта всех слов"""
        sorted_words = sorted(self._temp_word_counts.items(),
                            key=lambda x: x[1],
                            reverse=True)

        for word, count in sorted_words[:MAX_VOCAB_SIZE - 4]:
            if word not in self.word2index:

                if is_text:
                    if count > 10:
                        self.word2index[word] = self.n_words
                        self.word2count[word] = count
                        self.index2word[self.n_words] = word
                        self.n_words += 1
                    else:
                        self.word2count["<unk>"] += count

                else:
                    if count > 5:
                        self.word2index[word] = self.n_words
                        self.word2count[word] = count
                        self.index2word[self.n_words] = word
                        self.n_words += 1
                    else:
                        self.word2count["<unk>"] += count

        for word, count in sorted_words[MAX_VOCAB_SIZE - 4:]:
            self.word2count["<unk>"] += count

    def word_to_index(self, word: str) -> int:
        """Возвращает индекс слова или <unk>"""
        return self.word2index.get(word, self.word2index["<unk>"])

    def index_to_word(self, index: int) -> str:
        """Возвращает слово по индексу"""
        return self.index2word.get(index, self.word2index["<unk>"])

    def save(self, file_path: str):
        """Сохраняет словарь в файл"""
        with open(file_path, 'wb') as f:
            pickle.dump({
                'name': self.name,
                'word2index': self.word2index,
                'word2count': self.word2count,
                'index2word': self.index2word,
                'n_words': self.n_words
            }, f)

    @classmethod
    def load(cls, file_path: str):
        """Загружает словарь из файла"""
        with open(file_path, 'rb') as f:
            data = pickle.load(f)

        vocab = cls(data['name'])
        vocab.word2index = data['word2index']
        vocab.word2count = data['word2count']
        vocab.index2word = data['index2word']
        vocab.n_words = data['n_words']

        return vocab

    def __str__(self):
        """Строковое представление словаря"""
        return (
            f"Vocab(name='{self.name}', "
            f"n_words={self.n_words}, "
        )

    def __len__(self):
        return len(self.word2index)

In [7]:
input_vocab = Vocab("input")
target_vocab = Vocab("target")

for title, text in train_pairs:
    input_vocab.addText(text)
    target_vocab.addText(title)

input_vocab.build_vocab(is_text=True)
input_vocab.save("../../data/vocabs/src_vocab.pkl")
target_vocab.build_vocab(is_text=False)
target_vocab.save("../../data/vocabs/trg_vocab.pkl")


## Преобразование текста в датасет

In [9]:
def text_to_tensor(text: str, vocab: Vocab, add_sos_eos=True, max_len: int | None = None, truncate_from_start=False) -> torch.Tensor:
    """Преобразует текст в тензоры, с опциональной обрезкой"""
    tokens = text.strip().split()

    if max_len is not None:
        if truncate_from_start:
            tokens = tokens[-max_len:]
        else:
            tokens = tokens[:max_len]

    indices = [vocab.word_to_index(w) for w in tokens]

    if add_sos_eos:
        indices = [vocab.word2index["<sos>"]] + indices + [vocab.word2index["<eos>"]]

    return torch.tensor(indices, dtype=torch.long)


In [10]:
class TitleDataset(Dataset):
    def __init__(self, pairs: list[tuple[str, str]], input_vocab: Vocab, output_vocab: Vocab):
        """
            pairs — список пар (название, текст),
            input_vocab - словарь с частотами слов из текстов,
            output_vocab - словарь с частотами слов из названий
        """
        self.pairs = pairs
        self.input_vocab = input_vocab
        self.output_vocab = output_vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        title, text = self.pairs[idx]
        input_tensor = text_to_tensor(text, self.input_vocab, add_sos_eos=False, max_len=300)
        target_tensor = text_to_tensor(title, self.output_vocab, add_sos_eos=False, max_len=30)
        return input_tensor, target_tensor


In [11]:
def collate_fn(batch: List[tuple[str, str]]):
    """
    batch: list of (input_tensor, target_tensor)
    Returns:
        input_padded: [batch, src_len]
        target_padded: [batch, trg_len]
    """
    src_batch, trg_batch = zip(*batch)

    src_padded = pad_sequence(src_batch, padding_value=0, batch_first=True)
    trg_padded = pad_sequence(trg_batch, padding_value=0, batch_first=True)

    return src_padded, trg_padded


# Модель seq2seq

## Энкодер для seq2seq

In [12]:
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.3):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.lstm(embedded)
        return output, (hidden, cell)

## Декодер для seq2seq

In [13]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)

        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(0)
        embedded = self.dropout(self.embedding(input))
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        prediction = self.fc_out(output.squeeze(0))
        return prediction, hidden, cell

## Модель

In [42]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        assert encoder.hidden_size == decoder.hid_dim, "Hidden dimensions must match!"
        assert decoder.n_layers == 1, "Encoder must produce compatible layers for decoder"

    def forward(self, src, trg, teacher_forcing_ratio=0.1):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        encoder_outputs, (hidden, cell) = self.encoder(src)

        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs

    def train_epoch(self, dataloader, optimizer, criterion, clip=1.0):
        self.train()
        epoch_loss = 0
        total_grad_norm = 0
        batch_count = 0

        for src, trg in dataloader:
            src = src.to(self.device)
            trg = trg.to(self.device)
            optimizer.zero_grad()
            outputs = self(src, trg)
            output_dim = outputs.shape[-1]
            outputs = outputs[:, 1:].reshape(-1, output_dim)
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(outputs, trg)
            loss.backward()

            current_grad_norm = 0
            non_zero_grads = 0
            for p in self.parameters():
                if p.grad is not None:
                    grad_mean = p.grad.abs().mean()
                    if grad_mean < 0.01:
                        p.grad *= 2.0

                    current_grad_norm += p.grad.norm().item()
                    non_zero_grads += 1

            avg_grad_norm = current_grad_norm / max(1, non_zero_grads)
            dynamic_clip = min(clip, avg_grad_norm * 1.5)

            torch.nn.utils.clip_grad_norm_(self.parameters(), dynamic_clip)

            optimizer.step()

            epoch_loss += loss.item()
            total_grad_norm += current_grad_norm
            batch_count += 1

        return epoch_loss / len(dataloader)

    def evaluate(self, dataloader, criterion):
        self.eval()
        epoch_loss = 0

        with torch.no_grad():
            for src, trg in dataloader:
                src = src.to(self.device)
                trg = trg.to(self.device)

                output = self(src, trg, teacher_forcing_ratio=0.0)

                output_dim = output.shape[-1]
                output = output[:, 1:].reshape(-1, output_dim)
                trg = trg[:, 1:].reshape(-1)

                loss = criterion(output, trg)
                epoch_loss += loss.item()

        return epoch_loss / len(dataloader)

    def fit(self, train_loader, val_loader, optimizer, criterion, scheduler,
            num_epochs=10, clip=1.0, early_stopping_patience=3, model_save_path='models/best_model_seq2seq.pt'):
        """Полный цикл обучения модели с ранним остановом"""
        best_val_loss = float('inf')
        epochs_without_improvement = 0
        previous_val_loss = None

        for epoch in range(1, num_epochs + 1):
            start_time = time.time()
            train_loss = self.train_epoch(train_loader, optimizer, criterion, clip)
            val_loss = self.evaluate(val_loader, criterion)
            epoch_time = time.time() - start_time

            scheduler.step(val_loss)

            if previous_val_loss is not None and (abs(val_loss - previous_val_loss) <= 0.01 or val_loss > previous_val_loss):
                epochs_without_improvement += 1
            else:
                epochs_without_improvement = 0

            previous_val_loss = val_loss

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(self.state_dict(), model_save_path)

            if epochs_without_improvement >= early_stopping_patience:
                print(f"Ранний останов после {epoch:02} эпох (val_loss изменяется менее чем на ±0.01 в течение {early_stopping_patience} эпох)!")
                break

            print(
                f"{epochs_without_improvement}\n"
                f"Epoch {epoch:02} | Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f} "
                f"| LR: {optimizer.param_groups[0]['lr']:.6f} | Time: {epoch_time:.2f}s"
            )

        self.load_state_dict(torch.load(model_save_path))
        return best_val_loss

    def generate_sequence(self, src_sequence, src_vocab, trg_vocab, max_len=20):
        """Генерация последовательности по входным данным"""
        self.eval()

        if isinstance(src_sequence, str):
            tokens = nltk.word_tokenize(src_sequence.lower())
        else:
            tokens = src_sequence

        src_indexes = [src_vocab.word2index.get(token, src_vocab.word2index['<unk>']) for token in tokens]
        print(src_indexes)
        src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(self.device)

        with torch.no_grad():
            encoder_outputs, (hidden, cell) = self.encoder(src_tensor)

        trg_indexes = [trg_vocab.word2index['<sos>']]

        for _ in range(max_len):
            trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(self.device)

            with torch.no_grad():
                output, hidden, cell = self.decoder(trg_tensor, hidden, cell)

            pred_token = output.argmax(1).item()
            trg_indexes.append(pred_token)

            if pred_token == trg_vocab.word2index.get('<eos>', -1):
                break

        trg_tokens = []
        for idx in trg_indexes[1:]:
            token = trg_vocab.index2word.get(idx, "<unk>")
            if token != "eos":
                trg_tokens.append(token)

        return trg_tokens

    def calculate_bleu(self, dataloader, src_vocab, trg_vocab, max_len=20):
        """Вычисление BLEU score для DataLoader"""
        self.eval()
        references = []
        hypotheses = []
        smoothing = SmoothingFunction().method4

        with torch.no_grad():
            for src, trg in dataloader:
                src = src.to(self.device)
                trg = trg.to(self.device)

                output = self(src, trg, teacher_forcing_ratio=0.0)
                output = output.argmax(dim=-1)

                for i in range(trg.size(0)):
                    ref_indices = trg[i].cpu().numpy()
                    ref_tokens = []
                    for idx in ref_indices:
                        token = trg_vocab.index2word.get(int(idx), '<unk>')
                        if token not in ['sos', 'eos', '<pad>']:
                            ref_tokens.append(token)

                    hyp_indices = output[i].cpu().numpy()
                    hyp_tokens = []
                    for idx in hyp_indices:
                        token = trg_vocab.index2word.get(int(idx), '<unk>')
                        if token == 'eos':
                            break
                        if token not in ['sos', '<pad>']:
                            hyp_tokens.append(token)

                    references.append([ref_tokens])
                    hypotheses.append(hyp_tokens)

        return corpus_bleu(references, hypotheses, smoothing_function=smoothing)



вайбкодинг

In [15]:
PAD_IDX = 0
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [16]:
INPUT_DIM = input_vocab.n_words
OUTPUT_DIM = target_vocab.n_words
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 512
EMB_DIM = 128
ENC_DROPOUT = 0.4
DEC_DROPOUT = 0.4
N_LAYERS = 1

In [17]:
encoder = EncoderLSTM(INPUT_DIM, HID_DIM, dropout_p=ENC_DROPOUT)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(encoder, decoder, device).to(device)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn(


In [18]:
train_dataset = TitleDataset(train_pairs, input_vocab, target_vocab)
val_dataset = TitleDataset(val_pairs, input_vocab, target_vocab)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [21]:
AD_IDX = target_vocab.word2index["<pad>"]
num_epochs = 10

criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX, label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.3,
    patience=1,
    threshold=0.01
)

best_val_loss = model.fit(
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    num_epochs=10,
    clip=1.0,
    early_stopping_patience=1,
    model_save_path='../../models/best_model_seq2seq.pt'
)

0
Epoch 01 | Train Loss: 4.657 | Val Loss: 4.416 | LR: 0.000300 | Time: 137.416s
0
Epoch 02 | Train Loss: 4.490 | Val Loss: 4.367 | LR: 0.000300 | Time: 143.363s
0
Epoch 03 | Train Loss: 4.423 | Val Loss: 4.319 | LR: 0.000300 | Time: 146.269s
Ранний останов после 04 эпох (val_loss изменяется менее чем на ±0.01 в течение 1 эпох)!


In [51]:
model = Seq2Seq(encoder, decoder, device).to(device)

In [52]:
model.load_state_dict(torch.load("../../models/best_model_seq2seq.pt"))

<All keys matched successfully>

In [53]:
model

Seq2Seq(
  (encoder): EncoderLSTM(
    (embedding): Embedding(30000, 512)
    (lstm): LSTM(512, 512, batch_first=True)
    (dropout): Dropout(p=0.4, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1260, 128)
    (rnn): LSTM(128, 512, dropout=0.4)
    (fc_out): Linear(in_features=512, out_features=1260, bias=True)
    (dropout): Dropout(p=0.4, inplace=False)
  )
)

In [46]:
model = torch.load('../../models/best_model_seq2seq.pt')

In [48]:
model

OrderedDict([('encoder.embedding.weight',
              tensor([[ 0.9043,  1.2703,  0.7927,  ...,  0.9298,  0.4948, -0.9428],
                      [-0.4735,  1.2940, -1.3461,  ..., -0.2412, -0.8427, -1.2836],
                      [-0.4847, -0.5220,  0.0738,  ..., -1.3577,  1.1459, -1.8512],
                      ...,
                      [ 0.9665, -1.0452,  0.3203,  ...,  0.1029,  0.5494, -0.0333],
                      [-0.7574,  1.0757,  0.3603,  ...,  1.0432,  1.2531,  1.6537],
                      [-1.1803, -1.1679, -0.4149,  ..., -1.4314, -0.0824, -0.8447]])),
             ('encoder.lstm.weight_ih_l0',
              tensor([[-0.0067,  0.0259, -0.0045,  ..., -0.0115,  0.0413, -0.0193],
                      [-0.0659,  0.0205, -0.0887,  ..., -0.0139, -0.0410, -0.0249],
                      [-0.0602, -0.0203, -0.0176,  ...,  0.0026, -0.0741, -0.0297],
                      ...,
                      [ 0.0097,  0.0164,  0.0321,  ...,  0.0118,  0.0373,  0.0354],
                  

In [26]:
text = input()

In [36]:
from nltk.corpus import stopwords
from pymorphy3 import MorphAnalyzer

nltk.download("punkt")
nltk.download('stopwords')
nltk.download('punkt_tab')

morph = MorphAnalyzer()
stop_words = stopwords.words('russian')
extra_stopwords = ['это', 'который', 'весь', 'свой', 'такой', 'тем', 'чтобы']
stop_words.extend(extra_stopwords)
stop_words = set(stop_words)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/veronika_steklo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [37]:
from functools import lru_cache


@lru_cache(maxsize=10000)
def normalize_word(word: str) -> str:
    return morph.parse(word)[0].normal_form.replace('ё', 'е')

def normalization(text: List[str]) -> List[str]:
    return [normalize_word(word) for word in text]

In [38]:
from nltk import word_tokenize


def normalize_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    tokens = word_tokenize(text)
    tokens = normalization(tokens)
    return [token for token in tokens if token not in stop_words]


In [39]:
text

'— На днях в Лондоне, — продолжал он, — молодая девушка села в кэб. Она ехала встречать мать, с которой не виделась много лет. На углу какой-то улицы оглобля повозки разбивает в мелкие осколки окна кэба, длинный, как игла, осколок разбитого стекла пронзает сердце девушки. Она тут же умирает. Репортер называет это трагической смертью. Это неверно. Это не соответствует моим определениям сострадания и страха.\nЧувство трагического, по сути дела, — это лицо, обращенное в обе стороны, к страху и к состраданию, каждая из которых — его фаза. Ты заметил, я употребил слово «останавливает». Тем самым я подчеркиваю, что трагическая эмоция статична. Вернее, драматическая эмоция. Чувства, возбуждаемые неподлинным искусством, кинетичны: это влечение и отвращение. Влечение побуждает нас приблизиться, овладеть. Отвращение побуждает покинуть, отвергнуть. Искусства, вызывающие эти чувства, — порнография и дидактика — неподлинные искусства. Таким образом, эстетическое чувство статично. Мысль останавливае

In [40]:
print(normalize_text(text))

['день', 'лондон', 'продолжать', 'молодой', 'девушка', 'село', 'кэб', 'ехать', 'встречать', 'мать', 'видеться', 'год', 'угол', 'какойтый', 'улица', 'оглобля', 'повозка', 'разбивать', 'мелкий', 'осколок', 'окно', 'кэб', 'длинный', 'игла', 'осколок', 'разбитый', 'стекло', 'пронзать', 'сердце', 'девушка', 'умирать', 'репортер', 'называть', 'трагический', 'смерть', 'неверно', 'соответствовать', 'определение', 'сострадание', 'страх', 'чувство', 'трагический', 'суть', 'дело', 'лицо', 'обратить', 'оба', 'сторона', 'страх', 'сострадание', 'каждый', 'фаза', 'заметить', 'употребить', 'слово', 'останавливать', 'самый', 'подчеркивать', 'трагический', 'эмоция', 'статичный', 'верный', 'драматический', 'эмоция', 'чувство', 'возбуждать', 'неподлинный', 'искусство', 'кинетичный', 'влечение', 'отвращение', 'влечение', 'побуждать', 'приблизиться', 'овладеть', 'отвращение', 'побуждать', 'покинуть', 'отвергнуть', 'искусство', 'вызывающий', 'чувство', 'порнография', 'дидактик', 'неподлинный', 'искусство', '

In [47]:
model.generate_sequence(normalize_text(text), src_vocab=input_vocab, trg_vocab=target_vocab)

AttributeError: 'collections.OrderedDict' object has no attribute 'generate_sequence'

In [23]:
bleu_score = model.calculate_bleu(val_loader, input_vocab, target_vocab)
print(f'Validation BLEU score: {bleu_score*100:.2f}')

Validation BLEU score: 4.37


In [24]:
def generate_title(model, input_text, input_vocab, target_vocab, max_len=50, device="cpu", temperature=0.7):
    model.eval()

    tokens = re.findall(r"\w+|[.,!?;]", input_text.lower())
    src_indexes = [input_vocab.word2index.get(token, input_vocab.word2index["<unk>"]) for token in tokens]

    if not src_indexes:
        return "Невозможно проанализировать текст"

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    trg_indexes = [target_vocab.word2index["<sos>"]]

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(src_tensor, trg_tensor)

        output_dist = output[0,-1].div(temperature).exp()
        pred_token = torch.multinomial(output_dist, 1).item()

        if pred_token == target_vocab.word2index["<eos>"] or (i > 10 and len(set(trg_indexes[-5:])) < 2):
            break

        trg_indexes.append(pred_token)

    filtered = []
    for idx in trg_indexes[1:]:
        word = target_vocab.index_to_word(idx)
        if word not in ["<pad>", "<unk>", "<sos>", "<eos>"] and not word.isdigit():
            filtered.append(word)

    result = ' '.join(filtered).capitalize()
    result = re.sub(r'\s([?.!,](?:\s|$))', r'\1', result)

    return result

In [25]:
INPUT_DIM = input_vocab.n_words
OUTPUT_DIM = target_vocab.n_words
HID_DIM = 512
N_LAYERS = 1

encoder = EncoderLSTM(INPUT_DIM, HID_DIM, dropout_p=0.4)
decoder = Decoder(OUTPUT_DIM, 128, HID_DIM, N_LAYERS, 0.4)

model = Seq2Seq(encoder, decoder, device).to(device)
model.load_state_dict(torch.load('../../models/best_model_seq2seq.pt', map_location=device))
model.eval()

print("Генератор названий")
print("Введите текст (или 'выход' для завершения):")

while True:
    input_text = input("\n> ")

    if input_text.lower() in ['выход', 'exit', 'quit']:
        break

    if len(input_text.strip()) == 0:
        print("Пожалуйста, введите текст.")
        continue

    title = generate_title(model, input_text, input_vocab, target_vocab, device=device)
    print("\nСгенерированное название:")
    print(title)
    print("\nВведите следующий текст или 'выход' для завершения:")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn(


Генератор названий
Введите текст (или 'выход' для завершения):

Сгенерированное название:
Очень

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Город

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Цивилизация.

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Филип

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Мало

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Рай

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Знать.

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Дух в

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Сказка за

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Уже

Введите следующий текст или 'выход' для завершения:

Сгенерированное название:
Какой

Введите следующий текст или 'выход' 

KeyboardInterrupt: Interrupted by user

In [8]:
print("Размер словаря:", len(target_vocab))
print("Примеры слов:", [target_vocab.index_to_word(i) for i in range(10)])

Размер словаря: 1260
Примеры слов: ['<pad>', '<unk>', '<sos>', '<eos>', '.', ',', '...', 'и', 'в', '!']
